In [ ]:
!pip install dgl torch

In [ ]:
# dglを使ってデータをダウンロードする
import os

os.environ["DGLBACKEND"] = "pytorch"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import dgl
import dgl.data
from dgl.nn import GraphConv

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

In [ ]:
g = dataset[0]

In [ ]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

## DGLで学習

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [ ]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]

    for e in range(100):
        logits = model(g, features)
        pred = logits.argmax(1)

        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_TESt_acc = test_acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

## 独自モジュールで学習

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf
import numpy as np

import gnn

In [ ]:
adjacency_list = np.stack((g.edges()[0].numpy(), g.edges()[1].numpy()), axis=1)

In [ ]:
class GCN(tf.keras.Model):

    def __init__(self, h_feats: int, num_classes: int, adjacency_list: np.ndarray):
        super().__init__()
        self.conv1 = gnn.layers.GraphConv(h_feats, adjacency_list)
        self.conv2 = gnn.layers.GraphConv(num_classes, adjacency_list)
        
    def call(self, inputs, *args, **kwargs):
        x = self.conv1(inputs)
        x = self.conv2(inputs)
        return x